In [9]:
import json
import networkx as nx

## Random DAG

In [ ]:
# Create a random graph
G = nx.gnp_random_graph(30000, 0.1, seed=0, directed=True)

# Create a directed acyclic graph by only keeping edges that point from lower (newer) indices to higher (older)
# There's a directed edge between paper i and paper j if paper i cites paper j.
DAG = nx.DiGraph([(u,v) for (u,v) in G.edges() if u<v])
nx.is_directed_acyclic_graph(DAG)

## Construct network graph from Reddit data

Refer to http://snap.stanford.edu/data/web-RedditNetworks.html

In [10]:
# read in networks for subreddit /r/politics
with open("politics.json") as fp:
    month_nets = json.load(fp)

# see who the user sn00gan replied to in the first month
print(month_nets[0]["sn00gan"])

['TedTheGreek_Atheos', 'ptwonline', 'sn00gan', 'TiiziiO', 'caged_raptor']


In [22]:
# Create graph
G = nx.DiGraph()
for i in range(len(month_nets)):
    for k,v in month_nets[i].items():
        for vs in v:
            G.add_edge(k, vs)

In [26]:
len(G.edges)

1688750

In [27]:
len(G.nodes)

119780

## MIP Model

In [28]:
import gurobipy as gp
import random

In [32]:
# TODO: change p to be different for every edge and output a cascade subgraph
def independent_cascade(graph, seeds, p):
    """
    Runs the independent cascade model on the given graph with the given seed nodes and infection probability.
    
    Parameters:
    graph (networkx.Graph): The graph on which to run the model.
    seeds (list): The seed nodes from which the infection starts.
    p (float): The probability of a node getting infected from an infected neighbor.
    
    Returns:
    list: A list of nodes that got infected during the simulation.
    """
    # Set all nodes as not infected
    infected = set()
    # Add the seed nodes to the infected set
    infected.update(seeds)
    # Initialize the newly infected nodes list with the seed nodes
    newly_infected = set(seeds)
    
    # Run the model until there are no more newly infected nodes
    while newly_infected:
        # Get the neighbors of all newly infected nodes
        neighbors = set()
        for node in newly_infected:
            neighbors.update(set(graph.neighbors(node)))
        # Remove already infected and immune neighbors
        neighbors -= infected
        # Check for each neighbor if it gets infected
        newly_infected = set()
        for node in neighbors:
            if random.random() <= p:
                newly_infected.add(node)
        # Add newly infected nodes to the infected set
        infected.update(newly_infected)
    return list(infected)

In [31]:
def maximize_spread_cascades(graphs, budget, costs, seeds):
    """
    Solves the MIP model for maximizing the spread of cascades using network design as presented in the paper
    "Maximizing the Spread of Cascades Using Network Design" by Sheldon et al. for the given graph and budget.
    
    Parameters:
    graphs (list [networkx.Graph]): List of training cascades graphs, which are subgraphs of the original network
    budget (int): The budget of edges that can be added to the graph.
    costs (list): Cost of each action
    """
    # Initialize the MIP model
    model = gp.Model('maximize_spread_cascades')
    
    # Number of training cascades
    N = len(graphs)

    # Initialize the decision variables
    x = {}
    y = {}
    
    for k in range(N):
        for v in graph.edges:
            x[k, v] = model.addVar(vtype=GRB.COUTINUOUS, lb = 0, ub = 1, name=f'x_{k}_{v}')
    for v in graph.nodes:
        y[v] = model.addVar(vtype=GRB.BINARY, name=f'y_{v}')

    # Initialize the objective function to maximize the expected spread
    obj = 1/N * gp.quicksum(x[k, v] for k in range(N) for v in graphs[k].nodes)
    model.setObjective(obj, GRB.MAXIMIZE)

    # Add the budget constraint
    model.addConstr(gp.quicksum(cost[v]*y[v] for v in graphs[k].nodes) <= budget, name='budget_constraint')

    # Add the coverage constraint
    for v in graph.nodes:
        model.addConstr((x[k, v] <= y[v] for v in graphs[k].nodes for k in range(N)), name=f'coverage_constraint_{k}_{v}')

    # Add the edge constraints
    for u, v in graph.edges:
        model.addConstr(x[k, v] <= gp.quicksum(x[k, u] for u in graph[k].neighbors(v) 
                                               for k in range(N) for v in (set(graph[k].nodes) - set(seeds))), name=f'edge_constraint_{u}_{v}')

    # Optimize the model
    model.optimize()